In [1]:
#Mocap Arena Camera Calibrate Program
#Mateus Pincho - calibrate intrisics
# check pytorch and pytorch.vision versions
import torch
import torchvision
from glob import glob
import numpy as np
import cv2
from math import sqrt

print("PyTorch version : ", torch.__version__)
print("\nTorchVisoion Version : ", torchvision.__version__)

# Load the model avaliable in hubconf
calibrator = torch.hub.load('AlanSavio25/DeepSingleImageCalibration', 'calibrator')

PyTorch version :  2.3.1+cpu

TorchVisoion Version :  0.18.1+cpu


Using cache found in C:\Users\veloc/.cache\torch\hub\AlanSavio25_DeepSingleImageCalibration_master


In [117]:
# Define the image set for calibration
dataset    =  ['../../images/virtual/set_5/*.jpg',  # dataset[0]
              '../../images/virtual/set_10/*.jpg',  # dataset[1]
              '../../images/virtual/set_15/*.jpg',  # dataset[2]
              '../../images/virtual/set_20/*.jpg',  # dataset[3]
              '../../images/virtual/set_25/*.jpg',  # dataset[4]
              '../../images/chess/opencv/*.jpg']    # dataset[5]

image_set = glob(dataset[5])

In [118]:
focal_length = []
for image in image_set:

  results = calibrator.calibrate_from_path(image_path = image)
  focal_length.append(results['focal_length_pixels'])

print(focal_length)

[421.33416748046875, 340.64227294921875, 384.0802917480469, 421.33416748046875, 384.0802917480469, 464.32843017578125, 481.3655090332031, 464.32843017578125, 404.74224853515625, 404.74224853515625, 421.33416748046875, 445.1077880859375, 404.74224853515625, 404.74224853515625, 438.9750061035156, 464.32843017578125, 464.32843017578125, 404.74224853515625, 421.33416748046875, 384.0802917480469, 355.81463623046875, 353.591552734375, 421.33416748046875, 421.33416748046875, 404.74224853515625, 438.9750061035156]


In [131]:
f = np.median(focal_length)
resolution = (640,480)
o_x, o_y = resolution[0]/2, resolution[1]/2

camera_matrix = np.array([[  f,   0, o_x],
                          [  0,   f, o_y],
                          [  0,   0,   1]])
patternSize = (9,6)
rvecs = []
tvecs = []
errors = []
errors_rms = []

def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize
    return X

boardPoints = construct3DPoints(patternSize=(9,6), squareSize = 30)

print(camera_matrix)

[[421.33416748   0.         320.        ]
 [  0.         421.33416748 240.        ]
 [  0.           0.           1.        ]]


In [132]:
for image in image_set:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
    if ret == True:
        _, rvec, tvec = cv2.solvePnP(boardPoints, corners, camera_matrix, 0)
        rvecs.append(rvec)
        tvecs.append(tvecs)

        image_points_deep, _ = cv2.projectPoints(boardPoints, rvec, tvec, camera_matrix, 0)
        error = cv2.norm(corners, image_points_deep, normType= cv2.NORM_L2) / len(image_points_deep)
        errors.append(error)

        print(f"Error of Image {image}:", error, "px")


print("Median error: ", np.mean(errors))

Error of Image ../../images/chess/opencv/right11.jpg: 0.7590780509490123 px


Error of Image ../../images/chess/opencv/right13.jpg: 0.3339468147083239 px
Error of Image ../../images/chess/opencv/right03.jpg: 0.29805012365258915 px
Error of Image ../../images/chess/opencv/right12.jpg: 0.4410758759750347 px
Error of Image ../../images/chess/opencv/right02.jpg: 0.49665529793366153 px
Error of Image ../../images/chess/opencv/left11.jpg: 0.24067676983311406 px
Error of Image ../../images/chess/opencv/left13.jpg: 0.3067640402420952 px
Error of Image ../../images/chess/opencv/left12.jpg: 0.31391288352178653 px
Error of Image ../../images/chess/opencv/left09.jpg: 0.23389416514097594 px
Error of Image ../../images/chess/opencv/left14.jpg: 0.24535705651003095 px
Error of Image ../../images/chess/opencv/left02.jpg: 0.6176696512115813 px
Error of Image ../../images/chess/opencv/left06.jpg: 0.46556730885462083 px
Error of Image ../../images/chess/opencv/right09.jpg: 0.5462788199976955 px
Error of Image ../../images/chess/opencv/right06.jpg: 0.30461479165166616 px
Error of Im

In [134]:
for image in image_set:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)

    if ret == True:
        _, rvec, tvec = cv2.solvePnP(boardPoints, corners, camera_matrix, 0)
        rvecs.append(rvec)
        tvecs.append(tvecs)

        image_points_deep, _ = cv2.projectPoints(boardPoints, rvec, tvec, camera_matrix, 0)

        error_rms = cv2.norm(corners, image_points_deep, normType= cv2.NORM_L2) / sqrt(len(image_points_deep))
        errors_rms.append(error_rms)

        print(f" rms Error of Image {image}:", error_rms, "px")

print("Median error: ", np.median(errors_rms))

 rms Error of Image ../../images/chess/opencv/right11.jpg: 5.578061699314357 px
 rms Error of Image ../../images/chess/opencv/right13.jpg: 2.4539978917894616 px
 rms Error of Image ../../images/chess/opencv/right03.jpg: 2.1902121621668247 px
 rms Error of Image ../../images/chess/opencv/right12.jpg: 3.2412325019698582 px
 rms Error of Image ../../images/chess/opencv/right02.jpg: 3.6496561739622817 px
 rms Error of Image ../../images/chess/opencv/left11.jpg: 1.768605837097202 px
 rms Error of Image ../../images/chess/opencv/left13.jpg: 2.2542461100832147 px
 rms Error of Image ../../images/chess/opencv/left12.jpg: 2.30677916494232 px
 rms Error of Image ../../images/chess/opencv/left09.jpg: 1.718764075228966 px
 rms Error of Image ../../images/chess/opencv/left14.jpg: 1.8029987797223803 px
 rms Error of Image ../../images/chess/opencv/left02.jpg: 4.538926425213695 px
 rms Error of Image ../../images/chess/opencv/left06.jpg: 3.4212070428436845 px
 rms Error of Image ../../images/chess/op